In [115]:
from model_linear import *

import os
import time
import pickle

In [116]:
# check the stein/options to see all possible choices
options["type_optimization"] = "gradientDescent"
options["is_projection"] = False
options["tol_projection"] = 1.e-4
options["is_precondition"] = False
options["type_approximation"] = "fisher"
options["coefficient_dimension"] = 10

options["number_particles"] = 16
options["number_particles_add"] = 0
options["add_number"] = 0
options["add_step"] = 5
options["add_rule"] = 1

options["type_scaling"] = 1
options["type_metric"] = "prior"
options["kernel_vectorized"] = False

options["WGF"] = False

options["type_Hessian"] = "lumped"
options["low_rank_Hessian"] = False
options["rank_Hessian"] = 20
options["rank_Hessian_tol"] = 1.e-4
options["low_rank_Hessian_average"] = False
options["rank_Hessian_average"] = 20
options["rank_Hessian_average_tol"] = 1.e-4
options["gauss_newton_approx"] = True  # if error of unable to solve linear system occurs, use True

options["max_iter"] = 200
options["step_tolerance"] = 1e-7
options["step_projection_tolerance"] = 1e-6
options["line_search"] = True
options["search_size"] = 1e-1
options["max_backtracking_iter"] = 10
options["cg_coarse_tolerance"] = 0.5e-2
options["print_level"] = -1
options["plot"] = False

In [117]:
# generate particles
particle = Particle(model, options, comm)

In [118]:
filename = "data/laplace_nDimension_" + str(particle.particle_dimension) + ".p"
if os.path.isfile(filename):
    data_save = pickle.load(open(filename, 'rb'))
    mean = model.generate_vector(PARAMETER)
    mean.set_local(data_save["mean"])
    variance = model.generate_vector(PARAMETER)
    variance.set_local(data_save["variance"])
    particle.mean_posterior = mean
    particle.variance_posterior = variance
    particle.trace_posterior = data_save["true_statistics"][2]

# evaluate the variation (gradient, Hessian) of the negative log likelihood function at given particles
variation = Variation(model, particle, options, comm)

# evaluate the kernel and its gradient at given particles
kernel = Kernel(model, particle, variation, options, comm)

In [30]:
kernel.gradient_set

<dolfin.cpp.la.GenericVector; proxy of <Swig Object of type 'std::shared_ptr< dolfin::GenericVector > *' at 0x7f9ccc4b0a20> >

In [126]:
particle_local = np.concatenate([p.get_local().reshape([-1,1]) for p in particle.particles],1)

In [150]:
def calculate_BM_bandwidth(X, ibw_in, alpha, explore_ratio=1.1):
    N, d = X.shape
    Y = X+np.sqrt(2*alpha)*np.random.randn(N,d)
    X_sum = np.sum(X**2,axis=1)
    Dxx = -2*np.matmul(X, X.T)+X_sum.reshape([-1,1])+X_sum.reshape([1,-1])
    def get_obj_ibandw(bw):
        Kxy = np.exp(-Dxx*0.5/bw)
        dxKxy = np.matmul(Kxy,X)
        sumKxy = np.sum(Kxy,1).reshape([-1,1])
        xi = 1./bw*(dxKxy/sumKxy-X)
        X_new = X-alpha*xi
        res = MMD(X_new,Y,1.)
        return res
    obj_ibw_in = get_obj_ibandw(ibw_in)
    epsi = 1e-6
    grad_ibw_in = (get_obj_ibandw(ibw_in+epsi)-obj_ibw_in)/epsi
    if grad_ibw_in<0:
        ibw_1 = ibw_in*explore_ratio
    else:
        ibw_1 = ibw_in/explore_ratio
    obj_ibw_1 = get_obj_ibandw(ibw_1)
    slope_ibw = (obj_ibw_1-obj_ibw_in)/(ibw_1-ibw_in)
    ibw_2 = (ibw_in * slope_ibw - 0.5 * grad_ibw_in * (ibw_1 + ibw_in)) / (slope_ibw - grad_ibw_in)
    obj_ibw_2 = get_obj_ibandw(ibw_2)
    if not np.isnan(ibw_2) and ibw_2>0:
        if obj_ibw_1<obj_ibw_in:
            if obj_ibw_2<obj_ibw_1:
                ibw_out = ibw_2
            else:
                ibw_out = ibw_1
        else:
            if obj_ibw_2<obj_ibw_in:
                ibw_out = ibw_2
            else:
                ibw_out = ibw_in
    else:
        if obj_ibw_1<obj_ibw_in:
            ibw_out = ibw_1
        else:
            ibw_out = ibw_in
        
    return ibw_out

In [155]:
calculate_BM_bandwidth(particle_local,2,1)

2.2078003095095968

In [90]:
def MMD(X,Y,bandwidth=1.):
    # MMD based on Gaussian kernel
    # X, Y are with shapes N*d and M*d
    N, d = X.shape
    M, _ = Y.shape
    h = bandwidth
    X_sum = np.sum(X**2,axis=1)
    Y_sum = np.sum(Y**2,axis=1)
    Dxx = -2*np.matmul(X, X.T)+X_sum.reshape([-1,1])+X_sum.reshape([1,-1])
    Dxy = -2*np.matmul(X, Y.T)+X_sum.reshape([-1,1])+Y_sum.reshape([1,-1])
    Dyy = -2*np.matmul(Y, Y.T)+Y_sum.reshape([-1,1])+Y_sum.reshape([1,-1])
    Kxx = np.exp(-Dxx/(2*h))
    Kxy = np.exp(-Dxy/(2*h))
    Kyy = np.exp(-Dyy/(2*h))
    print(np.sum(Kxy.ravel()))
    res = 1./N**2*np.sum(Kxx.ravel())-2/(M*N)*np.sum(Kxy.ravel())+1./M**2*np.sum(Kyy.ravel())
    res = res*(2*np.pi*h)**(-d/2)
    return res

In [91]:
X = 2*np.random.randn(5,4)
Y = np.random.randn(8,4)+1

In [92]:
res = MMD(X,X)

5.12154017453


In [86]:
X.ravel()

array([ 0.45758509,  0.52861056,  0.87451421,  4.51489687, -2.02097938,
       -0.04879334,  1.76415551,  0.00706727, -0.64875285, -0.92858687,
       -0.23446177, -0.16591782,  2.33144651, -0.39047869, -3.22194885,
        2.31358819,  0.33524119, -2.78566337, -2.23917796, -1.27085672])

In [129]:
np.isnan(np.nan)

True